In [6]:
# 이미지 특성과 음악 무드 카테고리 매핑룰
music_mood_mapping = {
    # 색상에 따른 매핑
    'Red': ['dramatic', 'romantic', 'powerful'],
    'Orange': ['happy', 'upbeat', 'energetic'],
    'Brown': ['mellow', 'melancholic', 'nature'],
    'Gold': ['party', 'epic', 'upbeat'],
    'Yellow': ['happy', 'energetic', 'fun'],
    'Lime': ['nature', 'energetic', 'fun'],
    'Pink': ['romantic', 'soft', 'happy'],
    'Green': ['nature', 'relaxing', 'meditative'],
    'Mint': ['film', 'calm', 'relaxing'],
    'SeaBlue': ['calm', 'relaxing', 'ambiental'],
    'Sky': ['dream', 'calm', 'hopeful'],
    'Blue': ['calm', 'ambiental', 'cool'],
    'Purple': ['emotional', 'dramatic', 'ambiental'],
    'Black': ['dark', 'heavy', 'dramatic'],
    'White': ['mellow', 'holiday', 'calm'],
    'Grey': ['melancholic', 'mellow', 'calm'],

    # 색조에 따른 매핑
    'Warm': ['happy', 'upbeat', 'romantic' ],
    'Cool': ['calm', 'relaxing', 'melancholic'],

    # 명도에 따른 매핑
    'Bright': ['happy', 'energetic', 'fun'],
    'Mid': ['meditative', 'mellow', 'relaxing'],
    'Dark': ['dramatic', 'melancholic', 'space']
}

In [7]:
# 사용된 무드테마의 수
all_moods = set(sum(music_mood_mapping.values(), []))
number_of_moods = len(all_moods)
print("테마수", number_of_moods)

테마수 26


In [8]:
# 각 카테고리별 무드 개수
num_color_moods = 3 * 16  # 각 색상에 3개의 무드, 총 17개의 색상
num_tone_moods = 3 * 2    # 각 색조에 3개의 무드, 총 2개의 색조 (Warm, Cool)
num_brightness_moods = 3 * 3  # 각 명도에 3개의 무드, 총 3개의 명도 (Bright, Mid, Dark)

# 총 경우의 수 계산
total_combinations = num_color_moods * num_tone_moods * num_brightness_moods
total_combinations

2592

In [9]:
import pandas as pd
data = pd.read_csv("analysis_results.csv")
data.columns

Index(['Filename', 'Warm', 'Cool', 'Bright', 'Mid', 'Dark', 'Red', 'Orange',
       'Brown', 'Gold', 'Yellow', 'Lime', 'Green', 'Mint', 'SeaBlue', 'Sky',
       'Blue', 'Purple', 'Pink', 'Black', 'White', 'Grey', 'Tone',
       'DominantColor', 'Brightness', 'mood_mapping_Rule'],
      dtype='object')

In [10]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.auto import tqdm
from sklearn.base import clone

# 데이터셋을 불러옵니다.
data = pd.read_csv("analysis_results.csv")

# 이미지 특성에 기반한 음악 무드 추천 함수
def recommend_music_mood(dominant_color, tone, brightness):
    recommended_moods = set()
    
    if dominant_color in music_mood_mapping:
        recommended_moods.update(music_mood_mapping[dominant_color])
    
    if tone in music_mood_mapping:
        recommended_moods.update(music_mood_mapping[tone])
    
    if brightness in music_mood_mapping:
        recommended_moods.update(music_mood_mapping[brightness])
    
    return list(recommended_moods)  # 세트를 리스트로 변환

# 데이터셋의 각 행에 대해 추천된 음악 무드를 계산하고 새로운 컬럼에 저장합니다.
data['mood_mapping_Rule'] = data.apply(
    lambda row: recommend_music_mood(row['DominantColor'], row['Tone'], row['Brightness']), 
    axis=1
)

# 멀티-라벨 이진화를 위해 리스트 형태의 데이터를 준비합니다.
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['mood_mapping_Rule'])

# 나머지 모델 훈련 과정은 기존과 동일하게 진행합니다.

# 데이터 저장
file_path = "analysis_results.csv"
data.to_csv(file_path, index=False)


In [13]:
# tqdm 진행상황확인 랜포학습
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import joblib
from tqdm.auto import tqdm
from sklearn.base import clone

# 데이터셋 불러오기
data = pd.read_csv(file_path)

# 모델 훈련을 위한 함수 정의
def train_model(X, y, model, n_estimators):
    models = [clone(model) for _ in range(n_estimators)]
    for i, m in tqdm(enumerate(models), total=n_estimators, desc="Training models"):
        m.fit(X, y)
    return models

# 특성과 타겟 변수 선택
X = data[[
        'Warm', 'Cool',
        'Bright', 'Mid', 'Dark',
        'Red', 'Orange', 'Brown', 'Gold', 'Yellow',
        'Lime', 'Pink', 'Green', 'Mint', 'SeaBlue',
        'Sky', 'Blue', 'Purple',
        "Black", "White", "Grey"
        ]]

y = data['mood_mapping_Rule'].apply(lambda x: x.split(','))  # 태그를 분리하여 리스트로 변환
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)  # 멀티-라벨 이진화

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 훈련
trained_models = train_model(X_train, y_train, model, model.n_estimators)

# 모델 저장 (주의: 이 경우 여러 모델이므로 저장 방식을 조정해야 함)
model_filename = 'Recommended_Moodes.pkl'
joblib.dump(trained_models, model_filename)

# mlb 객체 저장
mlb_filename = 'Recommended_Moodes_mlb.pkl'
joblib.dump(mlb, mlb_filename)

Training models: 100%|██████████| 100/100 [00:12<00:00,  7.96it/s]


['Recommended_Moodes_mlb.pkl']

In [ ]:
train_model.

In [40]:
import pickle

# 'your_file.pkl' 대신에 실제 파일 경로를 입력하세요.
file_path = r'C:\Users\gjaischool\Desktop\MetaWave\MetaWave\model\mlb.pkl'

# 파일을 읽기 모드로 열기
with open(file_path, 'rb') as file:
    # Pickle을 사용하여 데이터 로드
    data = pickle.load(file)

# 데이터의 일부를 확인
# 예: 데이터가 리스트라면 처음 몇 개의 요소를 출력
print(data[1:10])


[" 'calm'" " 'cool'}" " 'dramatic'" " 'dramatic'}" " 'dream'}"
 " 'emotional'" " 'energetic'" " 'epic'}" " 'film'"]


In [41]:
import pickle

# 'your_file.pkl' 대신에 실제 파일 경로를 입력하세요.
file_path = r"C:\Users\gjaischool\Desktop\MetaWave\MetaWave\model\model.pkl"

# 파일을 읽기 모드로 열기
with open(file_path, 'rb') as file:
    # Pickle을 사용하여 데이터 로드
    data = pickle.load(file)

# 데이터의 일부를 확인
# 예: 데이터가 리스트라면 처음 몇 개의 요소를 출력
print(data[0:])
print(data[1:])


['Warm' 'Cool' 'Bright' 'Mid' 'Dark' 'Red' 'Orange' 'Brown' 'Gold'
 'Yellow' 'Lime' 'Pink' 'Green' 'Mint' 'SeaBlue' 'Sky' 'Blue' 'Purple'
 'Black' 'White' 'Grey']
['Cool' 'Bright' 'Mid' 'Dark' 'Red' 'Orange' 'Brown' 'Gold' 'Yellow'
 'Lime' 'Pink' 'Green' 'Mint' 'SeaBlue' 'Sky' 'Blue' 'Purple' 'Black'
 'White' 'Grey']


In [24]:
# 파일분류
import pandas as pd
import os
import shutil

# 원본 이미지 폴더와 새로운 폴더 경로 정의
data = pd.read_csv("analysis_results.csv")
categorized_images = 'categorized_images'  # 분류된 이미지를 저장할 새 폴더 경로


# 각 이미지를 해당하는 폴더로 카피하는 로직 수행
for index, row in data.iterrows():
    # 분류 폴더명 생성 (Tone, DominantColor, Brightness를 기준으로)
    folder_name = f"{row['Mood']}"

    # 원본 파일 경로
    original_file_path = os.path.join(directory_path, row['Filename'])

    # 새 폴더 경로
    new_folder_path = os.path.join(categorized_images, folder_name)
    os.makedirs(new_folder_path, exist_ok=True)  # 폴더가 없으면 생성, 이미 있으면 무시

    # 이미지를 새 폴더로 카피
    csv_file_path = os.path.join(new_folder_path, row['Filename'])
    shutil.copy(original_file_path, csv_file_path)

KeyError: 'Mood'

In [ ]:
def determine_mood(row, rules):
    
    for mood, values in rules.items():
        if row['Tone'] == values[0] and row['DominantColor'] == values[1] and row['Brightness'] == values[2]:
            return mood
    return 'Undefined'  # 일치하는 분위기가 없는 경우

# 데이터셋의 각 행에 대해 분위기를 결정합니다.
data['Mood'] = data.apply(lambda row: determine_mood(row, music_mood_mapping), axis=1)